# Goal:
- This code reads and converts .pkl data into table format
- Create Database using postgreSQL
- Exports from table as .pkl

# Updates
- Add a final “proof” comparator
- Capture index during insert
- Export using original order

# Prerequisites
- **postgreSQL** - Download and install it on your system
- Download_Link -> https://www.postgresql.org/download/windows/
- during installation - create and save superuser, pwd - which you need latter
- **Python** > 3.9+

In [1]:
# =========================================
# STEP 0: Imports
# =========================================
import pickle
import json
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

# ---------------- CONFIG ---------------- #
PKL_PATH = "../data/ghazi/ENSG00000188938.pkl"

PG_USER = "postgres"
PG_PASSWORD = "manjoor123$ps"
PG_HOST = "localhost"
PG_PORT = "5432"
DB_NAME = "protein_db_test7"

TABLE_NAME = "protein_table_ensg00000188938_test7"

In [2]:
# =========================================
# STEP 1: Load dataset
# =========================================
with open(PKL_PATH, "rb") as f:
    data = pickle.load(f)

print("Total proteins:", len(data))
print("PDB files per protein:", [len(e.get("pdb_files", [])) for e in data])

Total proteins: 5
PDB files per protein: [810, 396, 779, 864, 802]


In [3]:
# =========================================
# STEP 2: Connect to DB (create if not exists)
# =========================================
# Admin connection to create DB
admin_engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/postgres",
    isolation_level="AUTOCOMMIT"
)

with admin_engine.connect() as conn:
    result = conn.execute(
        text("SELECT 1 FROM pg_database WHERE datname=:name"), {"name": DB_NAME}
    ).fetchone()
    if not result:
        conn.execute(text(f"CREATE DATABASE {DB_NAME}"))
        print(f"Database '{DB_NAME}' created.")
    else:
        print(f"Database '{DB_NAME}' already exists.")

# Connect to the target database
engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{DB_NAME}"
)

Database 'protein_db_test7' created.


In [4]:
# =========================================
# STEP 3: Create table (single-table with BYTEA[])
# =========================================
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    gene_id VARCHAR(15) NOT NULL,
    transcript_id VARCHAR(30) NOT NULL,
    protein_index INTEGER NOT NULL,
    sequence VARCHAR(400),
    exons JSONB,
    protein_coding BOOLEAN,
    nmd BOOLEAN,
    pdb_ids TEXT[],
    pdb_files BYTEA[],
    PRIMARY KEY (gene_id, transcript_id)
);
"""

with engine.begin() as conn:
    conn.execute(text(create_table_sql))

print(f"Table '{TABLE_NAME}' created or already exists.")

Table 'protein_table_ensg00000188938_test7' created or already exists.


# Insert rows/proteins + PDBs

In [5]:
# =========================================
# STEP 4: Insert proteins + PDBs
# =========================================
def insert_proteins_with_pdbs(engine, data, table_name):
    """
    Insert proteins with PDB files into the database.
    Preserves original PKL order using protein_index.
    """

    sql = f"""
    INSERT INTO {table_name} (
        gene_id, transcript_id, protein_index,
        sequence, exons, protein_coding, nmd,
        pdb_ids, pdb_files
    )
    VALUES (
        %(gene_id)s, %(transcript_id)s, %(protein_index)s,
        %(sequence)s, %(exons)s, %(protein_coding)s, %(nmd)s,
        %(pdb_ids)s, %(pdb_files)s
    )
    ON CONFLICT (gene_id, transcript_id) DO NOTHING;
    """

    conn = engine.raw_connection()
    try:
        cur = conn.cursor()
        total_pdbs = 0  # initialize PDB counter

        for protein_index, entry in enumerate(data):
            # Prepare PDB IDs and content
            pdb_ids = [p["pdb_id"] for p in entry.get("pdb_files", [])]
            pdb_files = [psycopg2.Binary(p["content"]) for p in entry.get("pdb_files", [])]

            total_pdbs += len(pdb_files)  # count total PDBs

            # Execute insert
            cur.execute(sql, {
                "gene_id": entry["gene_id"],
                "transcript_id": entry["transcript_id"],
                "protein_index": protein_index,
                "sequence": entry.get("sequence"),
                "exons": json.dumps(entry.get("exons", [])),
                "protein_coding": entry.get("protein_coding", False),
                "nmd": entry.get("nmd", False),
                "pdb_ids": pdb_ids,
                "pdb_files": pdb_files
            })

        conn.commit()

    finally:
        conn.close()
        
    print(f"✅ Inserted {len(data)} proteins with {total_pdbs} total PDB files.")


# Run insert
insert_proteins_with_pdbs(engine, data, TABLE_NAME)

✅ Inserted 5 proteins with 3651 total PDB files.


In [6]:
# =========================================
# STEP 5: Verify table content
# =========================================
df_preview = pd.read_sql(f"""
SELECT gene_id, transcript_id, cardinality(pdb_files) AS pdb_count
FROM {TABLE_NAME}
ORDER BY gene_id;
""", engine)

df_preview

,gene_id,transcript_id,pdb_count
0,ENSG00000188938,ENST00000649557,810
1,ENSG00000188938,ENST00000375412,396
2,ENSG00000188938,ENST00000423591,779
3,ENSG00000188938,ENST00000476484,864
4,ENSG00000188938,ENST00000428378,802


In [8]:
# =========================================
# STEP 6: Export full table back to .pkl
# =========================================
def export_table_to_pkl(engine, table_name, output_path):
    """
    Export entire table back to a .pkl file.
    Preserves original PKL order using protein_index.
    """

    # Read from DB using protein_index to preserve order
    df = pd.read_sql(
        f"""
        SELECT * FROM {table_name}
        ORDER BY protein_index
        """,
        engine
    )

    data_out = []

    for _, row in df.iterrows():
        # Integrity check: number of PDB IDs matches number of PDB files
        assert len(row["pdb_ids"] or []) == len(row["pdb_files"] or [])

        protein_entry = {
            "gene_id": row["gene_id"],
            "transcript_id": row["transcript_id"],
            "sequence": row["sequence"],
            "exons": row["exons"] or [],
            "protein_coding": row["protein_coding"],
            "nmd": row["nmd"],
            "pdb_files": []
        }

        # Reconstruct PDB files preserving original IDs and content
        for pdb_id, pdb_bytes in zip(row["pdb_ids"] or [], row["pdb_files"] or []):
            protein_entry["pdb_files"].append({
                "pdb_id": pdb_id,
                "content": bytes(pdb_bytes)
            })

        data_out.append(protein_entry)

    # Write back to PKL
    with open(output_path, "wb") as f:
        pickle.dump(data_out, f)

    print(f"✅ Exported {len(data_out)} proteins (order-preserved and identity-safe)")


# Example usage
OUTPUT_PKL = "../data/ghazi/ENSG00000188938_export_all_test7.pkl"
export_table_to_pkl(engine, TABLE_NAME, OUTPUT_PKL)

✅ Exported 5 proteins (order-preserved and identity-safe)


# END OF NOTE BOOK